# Archivo de ETL (Extract Transform and Load)

En este archivo se van a realizar las tareas necesarias para procesar los archivos de homicidios y lesiones, haciendo un acomodo de información con el objetivo de tener un archivo limpio y fácil de analizar.

Aprovechando que los archivos en su formato original están en Excel, nos dimos a la tarea de realizar cierta exploración de los datos, además, se revisó a detalle el diccionado de datos de cada archivo 

In [3]:
# Se importan las librerías necesarias de trabajo

import pandas as pd # Para trabajar con datasets
import numpy as np # Para realizar operaciones dentro de los datasets

In [4]:
# Comenzamos con la carga de los archivos de excel en un dataframe de pandas
# Se crea un objeto de tipo archivo para almacenar el archivo de Excel de homicidios
Archivo_Homicidios = xls_file = pd.ExcelFile("..\..\DataSets originales\homicidios.xlsx", engine='openpyxl')
df_Homicidios_Hechos = pd.read_excel(Archivo_Homicidios, "HECHOS") # Se crea el DF de homicidios de la pestaña de "Hecho"
df_Homicidios_Victimas = pd.read_excel(Archivo_Homicidios, "VICTIMAS") # Se crea el DF de homicidios de la pestaña "Victimas"
# Se crea un objeto de tipo archivo para almacenar el archivo de Excel de lesiones
Archivo_Lesiones = xls_file = pd.ExcelFile("..\..\DataSets originales\lesiones.xlsx", engine='openpyxl')
df_Lesiones_Hechos = pd.read_excel(Archivo_Lesiones, "HECHOS") # Se crea el DF de homicidios de la pestaña de "Hecho"
df_Lesiones_Victimas = pd.read_excel(Archivo_Lesiones, "VICTIMAS") # Se crea el DF de homicidios de la pestaña "Victimas"

# Limpieza de la pestaña hechos del archivo homicidios

In [ ]:
# Analizamos de forma general
df_Homicidios_Hechos.info()

In [5]:
# Haciendo uso del diccionario de datos y de la exploración de las colulmnas por medio de la aplicación Excel, 
# nos damos cuenta que podemos realizar ciertos arreglos a nuestro dataframe

# Se eliminan los espacios en blanco existentes en las columnas
df_Homicidios_Hechos.map(lambda x: x.strip() if isinstance(x, str) else x)
# Se reemplazan todos los valores que digan "Point (. .)" por NAN en la columna XY (CABA)
df_Homicidios_Hechos["XY (CABA)"].replace(["Point (..)", ""], np.nan, inplace=True)
# Se reemplazan todos los valores que digan "." en la columna pos x
df_Homicidios_Hechos["pos x"].replace([".", ""], np.nan, inplace=True)
# Se reemplazan todos los valores que digan "." en la columna pos y
df_Homicidios_Hechos["pos y"].replace([".", ""], np.nan, inplace=True)
# Se reemplazan todos los valores 'SD' por NaN en todo el DataFrame
df_Homicidios_Hechos.replace(['SD','sd'], np.nan, inplace=True)
# Se reemplazan todos los valores vacios por NaN en todo el dataframe
df_Homicidios_Hechos.replace([''], np.nan, inplace=True)


In [8]:
# Se procede a arreglar el tamaño del dataframe por medio del acomodo de sus columnas

# Se eliminan las columnas de año, mes, día y hora, debido a que ya se encuentran en otra columna con el nombre de fecha
df_Homicidios_Hechos.drop(["AAAA","MM","DD"], axis=1, inplace=True)
# Se renombran las columnas para leerlas con mayor facilidad
Columnas_Renombrar = {'ID': 'Id_Homicidio', 'N_VICTIMAS': 'Nro_Victimas', 'FECHA': 'Fecha', 'HORA': 'Hora_Hecho', 'HH': 'Hora',
               'TIPO_DE_CALLE': 'Tipo_Calle', "LUGAR_DEL_HECHO": "Lugar_Hecho", "Dirección Normalizada": "Direccion_Normalizada", 'COMUNA': 'Comuna', 'pos x': 'Longitud', 'pos y': 'Latitud',
               'VICTIMA': 'Vehiculo_Victima', 'ACUSADO': 'Vehiculo_Acusado', "PARTICIPANTES": "Participantes"} # Se define el diccionario de columnas 
df_Homicidios_Hechos.rename(columns=Columnas_Renombrar, inplace=True) # Se ejecuta el reemplazo en el DF
# Se convierten en enteros las columnas que contienen enteros
df_Homicidios_Hechos["Nro_Victimas"] = pd.to_numeric(df_Homicidios_Hechos["Nro_Victimas"], errors='coerce').astype('Int64')
df_Homicidios_Hechos['Hora'] = pd.to_numeric(df_Homicidios_Hechos['Hora'], errors='coerce').astype('Int64')
df_Homicidios_Hechos['Comuna'] = pd.to_numeric(df_Homicidios_Hechos['Comuna'], errors='coerce').astype('Int64')

Analizando valores específicos de cada columna y realizando cierta investigación sobre la manera en que está organizada la Ciudad Autónoma de Buenos Aires (CABA), notamos que se distribuye la población por medio de comunas, que van de la 1 a la 15, por ende, buscamos reemplazar aquellos valores que marquen un valor 0 en la comuna.

Adicional, aprovechando que la cantidad de registros con una Latitud y Longitud son pocos, nos dimos a la tarea de utilizar google maps para obtener estos valores y poder reemplazar los datos vacíos por datos reales.

Por último, notamos un error de escritura, donde uno de los registros muestra en la columna "Victima" el valor "Peaton-Moto", el cuál, al cruzarlo con la pestaña de Victimas, nos damos cuenta que el valor correcto tendrá que ser Peaton

In [10]:
# Comenzamos con los apartados de comuna, Longitud y Latitud

# Definir los registros que deseas reemplazar en un diccionario
registros_a_reemplazar = [
    {'Id_Homicidio': '2016-0174', 'Comuna': 7, 'Longitud': -58.369529920, 'Latitud': -34.622472560}, # Comunas 0 
    {'Id_Homicidio': '2016-0151', 'Comuna': 10, 'Longitud': -58.52756365, 'Latitud': -34.62772274}, # Comunas 0 
    {'Id_Homicidio': '2018-0039', 'Comuna': 14, 'Longitud': -58.457579360, 'Latitud': -34.538957930}, #sin Longitud/Latitud
    {'Id_Homicidio': '2020-0026', 'Comuna': 14, 'Longitud': -58.457579360, 'Latitud': -34.538957930}, #sin Longitud/Latitud  
    {'Id_Homicidio': '2017-0042', 'Comuna': 14, 'Longitud': -58.457579360, 'Latitud': -34.538957930}, #sin Longitud/Latitud  
    {'Id_Homicidio': '2017-0050', 'Comuna': 9, 'Longitud': -58.481800300, 'Latitud': -34.647005340}, #sin Longitud/Latitud
    {'Id_Homicidio': '2017-0051', 'Comuna': 7, 'Longitud': -58.479447760, 'Latitud': -34.678121010}, #sin Longitud/Latitud      
    {'Id_Homicidio': '2017-0140', 'Comuna': 4, 'Longitud': -58.380781350, 'Latitud': -34.622364700}, #sin Longitud/Latitud
    {'Id_Homicidio': '2020-0039', 'Comuna': 9, 'Longitud': -58.483096200, 'Latitud': -34.671405150}, #sin Longitud/Latitud   
    {'Id_Homicidio': '2021-0023', 'Comuna': 4, 'Longitud': -58.364905600, 'Latitud': -34.622974220}, #sin Longitud/Latitud  
    {'Id_Homicidio': '2016-0052', 'Comuna': 13, 'Longitud': -58.457579360, 'Latitud': -34.538957930}, #sin Longitud/Latitud   
    {'Id_Homicidio': '2016-0136', 'Comuna': 4, 'Longitud': -58.363401550, 'Latitud': -34.624371200}, #sin Longitud/Latitud  
] 

# Creamos un for para hacer un recorrido por todos los registros a reemplazar
for registro in registros_a_reemplazar: # Se inicia el ciclo y se guardarán en nuevas variables los datos que contiene el diccionario
    Id_Hecho = registro['Id_Homicidio'] 
    Comuna = registro['Comuna']
    Longitud = registro['Longitud']
    Latitud = registro['Latitud']

    # Se utiliza loc para reemplazar los valores específicos en cada registro según el Id_Hecho
    df_Homicidios_Hechos.loc[df_Homicidios_Hechos['Id_Homicidio'] == Id_Hecho, 'Comuna'] = Comuna
    df_Homicidios_Hechos.loc[df_Homicidios_Hechos['Id_Homicidio'] == Id_Hecho, 'Longitud'] = Longitud
    df_Homicidios_Hechos.loc[df_Homicidios_Hechos['Id_Homicidio'] == Id_Hecho, 'Latitud'] = Latitud

# Se hace la sistitución del valor "PeatonMoto" de la columna victima
df_Homicidios_Hechos.loc[df_Homicidios_Hechos['Vehiculo_Victima'] == "PEATON_MOTO", 'Vehiculo_Victima'] = "Peaton"

In [11]:
# Se termina la limpieza del df de homicidios en la pestaña de hechos
df_Homicidios_Hechos.head()

,Id_Homicidio,Nro_Victimas,Fecha,Hora_Hecho,Hora,Lugar_Hecho,Tipo_Calle,Calle,Altura,Cruce,Direccion_Normalizada,Comuna,XY (CABA),Longitud,Latitud,Participantes,Vehiculo_Victima,Vehiculo_Acusado
0,2016-0001,1,2016-01-01,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


# Limpieza de la pestaña Victimas del archivo homicidios

In [12]:
# Analizamos de forma general nuestro dataframe de victimas en homicidios
df_Homicidios_Victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [13]:
# Haciendo uso del diccionario de datos y de la exploración de las colulmnas por medio de la aplicación Excel, 
# nos damos cuenta que podemos realizar ciertos arreglos a nuestro dataframe

# Se eliminan los espacios en blanco existentes en las columnas
df_Homicidios_Victimas.map(lambda x: x.strip() if isinstance(x, str) else x)
# Se reemplazan todos los valores 'SD' por NaN en todo el DataFrame
df_Homicidios_Victimas.replace(['SD','sd'], np.nan, inplace=True)
# Se reemplazan todos los valores vacios por NaN en todo el dataframe
df_Homicidios_Victimas.replace([''], np.nan, inplace=True)

In [14]:
# Se procede a arreglar el tamaño del dataframe por medio del acomodo de sus columnas

# Se eliminan las columnas de fecha, año, mes, día, victima debido a que ya se encuentran en la pestaña de hechos
df_Homicidios_Victimas.drop(["FECHA","AAAA","MM","DD","VICTIMA"], axis=1, inplace=True)
# Se renombran las columnas para leerlas con mayor facilidad
Columnas_Renombrar = {'ID_hecho': 'Id_Homicidio', 'ROL': 'Rol', 'SEXO': 'Sexo', "EDAD": "Edad", "FECHA_FALLECIMIENTO": "Fecha_Fallecimiento"} # Se define el diccionario de columnas 
df_Homicidios_Victimas.rename(columns=Columnas_Renombrar, inplace=True) # Se ejecuta el reemplazo en el DF
# Se convierten en enteros las columnas que contienen enteros
df_Homicidios_Victimas["Edad"] = pd.to_numeric(df_Homicidios_Victimas["Edad"], errors='coerce').astype('Int64') 


# Limpieza del archivo Lesiones en la pestaña hechos

In [15]:
# Análizamos el dataframe de forma general
df_Lesiones_Hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [16]:
# Haciendo uso del diccionario de datos y de la exploración de las colulmnas por medio de la aplicación Excel, 
# nos damos cuenta que podemos realizar ciertos arreglos a nuestro dataframe

# Se eliminan los espacios en blanco existentes en las columnas
df_Lesiones_Hechos.map(lambda x: x.strip() if isinstance(x, str) else x)
# Se reemplazan todos los valores 'SD' por NaN en todo el DataFrame
df_Lesiones_Hechos.replace(['SD','sd'], np.nan, inplace=True)
# Se reemplazan todos los valores vacios por NaN en todo el dataframe
df_Lesiones_Hechos.replace([''], np.nan, inplace=True)

In [18]:
# Se sistituye el valor de aquellas comunas no especificadas 
df_Lesiones_Hechos.loc[df_Lesiones_Hechos['comuna'] == "No Especificada", 'comuna'] = np.nan
# Se sustituye el valor de las edades que marquen 0
df_Lesiones_Hechos.loc[df_Lesiones_Hechos['n_victimas'] == "0", 'n_victimas'] = np.nan

In [38]:
# Convertimos en enteros aquellas columnas que tienen números
df_Lesiones_Hechos["n_victimas"] = pd.to_numeric(df_Lesiones_Hechos["n_victimas"], errors='coerce').astype('Int64') 
df_Lesiones_Hechos["franja_hora"] = pd.to_numeric(df_Lesiones_Hechos["franja_hora"], errors='coerce').astype('Int64') 
df_Lesiones_Hechos["comuna"] = pd.to_numeric(df_Lesiones_Hechos["comuna"], errors='coerce').astype('Int64')

# Se cambia el nombre a la columna de ID
Columnas_Renombrar = {'id': 'Id_Lesion'} # Se define el diccionario de columnas 
df_Lesiones_Hechos.rename(columns=Columnas_Renombrar, inplace=True) # Se ejecuta el reemplazo en el DF

# Limpieza del archivo lesiones en la pestaña victimas

In [21]:
# Analizamos de forma general el DF
df_Lesiones_Victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


In [22]:
# Haciendo uso del diccionario de datos y de la exploración de las colulmnas por medio de la aplicación Excel, 
# nos damos cuenta que podemos realizar ciertos arreglos a nuestro dataframe

# Se eliminan los espacios en blanco existentes en las columnas
df_Lesiones_Victimas.map(lambda x: x.strip() if isinstance(x, str) else x)
# Se reemplazan todos los valores 'SD' por NaN en todo el DataFrame
df_Lesiones_Victimas.replace(['SD','sd'], np.nan, inplace=True)
# Se reemplazan todos los valores vacios por NaN en todo el dataframe
df_Lesiones_Victimas.replace([''], np.nan, inplace=True)

In [26]:
# Se eliminan las columnas de año, mes, día, vehículo_Victima y gravedad
df_Lesiones_Victimas.drop(["AAA","MM","DD","VEHICULO_VICTIMA","GRAVEDAD"], axis=1, inplace=True)


In [34]:
# Se sustituye el valor de los sexos para estandarizar
df_Lesiones_Victimas.loc[df_Lesiones_Victimas['SEXO'] == "Varon", 'SEXO'] = "MASCULINO"
df_Lesiones_Victimas.loc[df_Lesiones_Victimas['SEXO'] == "Mujer", 'SEXO'] = "FEMENINO"

# Se cambia el nombre a la columna de ID
Columnas_Renombrar = {'ID hecho': 'Id_Lesion'} # Se define el diccionario de columnas 
df_Lesiones_Victimas.rename(columns=Columnas_Renombrar, inplace=True) # Se ejecuta el reemplazo en el DF

# Union y definición final de archivos

Se procede a unir los dataframe de cada archivo y concentrar la información en un archivo por separado

In [39]:
# Unimos los dos dataframe de cada archivo en uno solo
Union_Homicidios = pd.merge(df_Homicidios_Hechos, df_Homicidios_Victimas, left_on='Id_Homicidio', right_on='Id_Homicidio', how='inner')
Union_Lesiones = pd.merge(df_Lesiones_Hechos, df_Lesiones_Victimas, left_on='Id_Lesion', right_on='Id_Lesion', how='inner')

# Almacenar los DataFrames finales en un nuevo archivo CSV

Union_Homicidios.to_csv('..\..\DataSets Limpios\Homicidios_Limpio.csv', index=False)
Union_Lesiones.to_csv('..\..\DataSets Limpios\Lesiones_Limpio.csv', index=False)